In [ ]:
#@markdown #**IMPORTING NECESSARY LIBRARIES**
###1. Load Data and Splot Data

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import keras
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
#@markdown ##**Spliting the data into x_train, y_train, x_test and y_test**
# splitting the data into test and train set
(x_train, _), (x_test, _) = fashion_mnist.load_data()


In [ ]:
#@markdown ##**Displaying the sample images from x_train**
import matplotlib.pyplot as plt

# Number of digits to display
n = 10

# Create a figure to display the images
plt.figure(figsize=(20, 4))

# Loop through the first 'n' images
for i in range(n):
    # Create a subplot within the figure
    ax = plt.subplot(2, n, i + 1)

    # Display the original image
    plt.imshow(X_test[i].reshape(28, 28))

    # Set colormap to grayscale
    plt.gray()

    # Hide x-axis and y-axis labels and ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

# Show the figure with the images
plt.show()

# Close the figure
plt.close()


In [ ]:
#@markdown ##**Displaying the shape of x_train and X_test**

print(x_train.shape)
print(x_test.shape)

In [ ]:
 #@markdown #**IMPORTING LIBRARIES FOR CNN**
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from tensorflow.keras.models import Model

In [ ]:
 #@markdown #**SHAPE**
print (x_train.shape)
print (x_test.shape)

In [ ]:
 #@markdown #**Create Noisy Data**
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)
print(x_train_noisy.shape,x_test_noisy.shape )

In [ ]:
 #@markdown #**PRINTING NOISY DATA**
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(x_train_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
plt.close()

In [ ]:
 #@markdown #**ARCHITECTURE OF AUTOENCODER**
input_layer_cnv = Input(shape = (28,28,1))
ae_cnv_en = Conv2D (32, (3,3), activation = "relu", padding = "same", kernel_initializer="he_normal")(input_layer_cnv)
ae_cnv_en = MaxPooling2D ((2,2), padding="same")(ae_cnv_en)

ae_cnv_en = Conv2D (32, (3,3), activation = "relu", padding = "same")(ae_cnv_en)
ae_cnv_en = MaxPooling2D ((2,2), padding="same")(ae_cnv_en)

ae_cnv_en = Conv2D (4, (3,3), activation = "relu", padding = "same")(ae_cnv_en)
ae_cnv_en = MaxPooling2D ((2,2), padding="same")(ae_cnv_en)

ae_cnv_en = Flatten(name = "bot")(ae_cnv_en)

ae_cnv_de = Reshape((4,4,4), input_shape= (64,), name= "botnext0")(ae_cnv_en)

ae_cnv_de = Conv2D (4, (3,3), activation = "relu", padding = "same",name= "botnext1")(ae_cnv_de)
ae_cnv_de = UpSampling2D ((2,2),name= "botnext2")(ae_cnv_de)

ae_cnv_de = Conv2D (32, (3,3), activation = "relu", padding = "same", name= "botnext3")(ae_cnv_de)
ae_cnv_de = UpSampling2D ((2,2), name= "botnext4")(ae_cnv_de)

ae_cnv_de = Conv2D (32, (3,3), activation = "relu", padding = "valid",name= "botnext5")(ae_cnv_de)
ae_cnv_de = UpSampling2D ((2,2), name= "botnext6")(ae_cnv_de)

ae_cnv_de = Conv2D (1, (3,3), activation = "sigmoid", padding = "same",name = "botnext7")(ae_cnv_de)
Ae_Conv_denoise = Model (inputs = input_layer_cnv, outputs = ae_cnv_de)

#Ae_Conv.compile(optimizer=tf.keras.optimizers.SGD(0.09,clipvalue=2.5), loss='binary_crossentropy', metrics = ["accuracy"])

Ae_Conv_denoise.compile(optimizer=tf.keras.optimizers.Adadelta(0.1,clipvalue=2), loss='binary_crossentropy', metrics = ["accuracy"])
Ae_Conv_denoise.summary()

In [ ]:
 #@markdown #**ENCODER**
ae_conv_dns_encoder = Model (inputs = input_layer_cnv,outputs = Ae_Conv_denoise.get_layer("bot").output, name = "Conv_AE_dns_encoder")
ae_conv_dns_encoder.summary()

In [ ]:
 #@markdown #**DECODER**
encode_inp_cnv = Input(shape = (64,))
tmp_dec = Ae_Conv_denoise.get_layer("botnext0")(encode_inp_cnv)
for i in range(1,8):
  st = "botnext{}".format(i)
  tmp_dec = Ae_Conv_denoise.get_layer(st)(tmp_dec)

ae_conv_dns_decoder = Model(inputs = encode_inp_cnv,outputs = tmp_dec, name= "Conv_AE_dns_decoder")
ae_conv_dns_decoder.summary()

In [ ]:
 #@markdown #**TRAINING**
from time import time
tic = time()
Ae_Conv_denoise.fit(x_train_noisy, x_train,
                epochs=500,
                verbose=2,
                batch_size=256,
                shuffle=False, validation_split = 0.1)
                #validation_data=(x_test, x_test))
toc = time()
print("Training Took {} Secs".format(toc-tic))

In [ ]:
 #@markdown #**TESTING**
dns_encoded_imgs = ae_conv_dns_encoder.predict(x_test_noisy)
#encoded_imgs = ae_conv_encoder.predict(x_train[0:100])
print (dns_encoded_imgs.shape)

In [ ]:
 #@markdown #**RECONSTRUCTED FROM NOISY IMAGES**
#decoded_imgs = ae_conv_decoder.predict(encoded_imgs)
dns_decoded_imgs = ae_conv_dns_decoder.predict(dns_encoded_imgs)
# dns_decoded_image = Ae_Conv_denoise.predict(x_test_noisy)
print("Recreated image Representation of Shape {} using Decoder and reduced Image representation of shape {}".format (dns_decoded_imgs.shape,
                                                                                                                      dns_encoded_imgs.shape))
n = 10  # how many digits we will display
k =16
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy[i*k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(dns_decoded_imgs[i*k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
 #@markdown #**RECONSTRUCTED FROM NOISY IMAGES**
dns_decoded_image = Ae_Conv_denoise.predict(x_test_noisy)
print("Recreated image Representation of Shape {} using Decoder and reduced Image representation of shape {}".format (dns_decoded_image.shape,
                                                                                                                      dns_decoded_image.shape))
n = 10  # how many digits we will display
k =16
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy[i*k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(dns_decoded_image[i*k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()